<a href="https://colab.research.google.com/github/sabaakram539/Book_Recommendation_System/blob/main/BookRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Through this project, I am going to do a project on Clustering. It will reccoomend the books to the users based on the books that users read. Data set is taken from Kaggle. Let's import the libraries and then the datasets.

In [2]:
import numpy as np
import pandas as pd

In [3]:
books=pd.read_csv("BX-Books.csv",sep=";",error_bad_lines=False,encoding="latin-1", engine="python")

Skipping line 815: ';' expected after '"'
Skipping line 1679: ';' expected after '"'
Skipping line 2531: ';' expected after '"'
Skipping line 2640: ';' expected after '"'
Skipping line 3236: ';' expected after '"'
Skipping line 3256: ';' expected after '"'
Skipping line 4906: ';' expected after '"'
Skipping line 6452: ';' expected after '"'
Skipping line 8349: ';' expected after '"'
Skipping line 9669: ';' expected after '"'
Skipping line 9723: ';' expected after '"'
Skipping line 10010: ';' expected after '"'
Skipping line 10192: ';' expected after '"'
Skipping line 10754: ';' expected after '"'
Skipping line 10824: ';' expected after '"'
Skipping line 10862: ';' expected after '"'
Skipping line 11442: ';' expected after '"'
Skipping line 11584: ';' expected after '"'
Skipping line 11859: ';' expected after '"'
Skipping line 12206: ';' expected after '"'
Skipping line 12229: ';' expected after '"'
Skipping line 12719: ';' expected after '"'
Skipping line 12885: ';' expected after '"'


In [4]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [5]:
books.shape

(29782, 8)

In [6]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [7]:
books.rename(columns={"Book-Title":"Title","Book-Author":"Author","Year-Of-Publication":"Year"},inplace=True)

In [8]:
books.head(2)

,ISBN,Title,Author,Year,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [9]:
books=books[["ISBN","Title","Author","Year","Publisher"]]
books.head(2)

,ISBN,Title,Author,Year,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada


In [10]:
users=pd.read_csv("BX-Users.csv",sep=";",error_bad_lines=False,encoding="latin-1",engine="python")

Skipping line 269: ';' expected after '"'
Skipping line 1104: ';' expected after '"'
Skipping line 1274: ';' expected after '"'
Skipping line 1307: ';' expected after '"'
Skipping line 1696: ';' expected after '"'
Skipping line 1703: ';' expected after '"'
Skipping line 1999: ';' expected after '"'
Skipping line 2300: ';' expected after '"'
Skipping line 2450: ';' expected after '"'
Skipping line 2577: ';' expected after '"'
Skipping line 2665: ';' expected after '"'
Skipping line 2713: ';' expected after '"'
Skipping line 2728: ';' expected after '"'
Skipping line 2818: ';' expected after '"'
Skipping line 2870: ';' expected after '"'
Skipping line 2940: ';' expected after '"'
Skipping line 2998: ';' expected after '"'
Skipping line 3404: ';' expected after '"'
Skipping line 3935: ';' expected after '"'
Skipping line 4342: ';' expected after '"'
Skipping line 4446: ';' expected after '"'
Skipping line 4641: ';' expected after '"'
Skipping line 4757: ';' expected after '"'
Skipping lin

In [11]:
users.head(2)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [12]:
ratings=pd.read_csv("BX-Book-Ratings.csv",sep=";",error_bad_lines=False,encoding="latin-1",engine="python")

Skipping line 21689: ';' expected after '"'
Skipping line 45329: ';' expected after '"'
Skipping line 45330: ';' expected after '"'
Skipping line 183262: ';' expected after '"'
Skipping line 210093: ';' expected after '"'


In [13]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [14]:
ratings.rename(columns={"Book-Rating":"Rating"},inplace=True)

In [15]:
ratings.head(2)

,User-ID,ISBN,Rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0


In [16]:
books.shape

(29782, 5)

In [17]:
users.shape

(195247, 3)

In [18]:
ratings.shape

(377276, 3)

Now after cleaning and reshuffling the data, I am going to form the book reccommendation system based on only those users who have given Ratings to atleast 50 books.

In [19]:
ratings.head(2)

,User-ID,ISBN,Rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0


In [20]:
ratings['User-ID'].value_counts()

11676     13600
35859      5850
278418     4533
76352      3367
16795      2948
          ...  
19689         1
35700         1
58233         1
52092         1
41081         1
Name: User-ID, Length: 35324, dtype: int64

In [21]:
ratings['User-ID'].value_counts().shape

(35324,)

In [22]:
x=ratings['User-ID'].value_counts()>=50 ## The user id of only those who have given to atleast 50 books.
x

11676      True
35859      True
278418     True
76352      True
16795      True
          ...  
19689     False
35700     False
58233     False
52092     False
41081     False
Name: User-ID, Length: 35324, dtype: bool

In [23]:
x[x].shape  # 608 users are those who have given ratings to atleast 50 books.

(1133,)

In [24]:
y=x[x].index
y

Int64Index([ 11676,  35859, 278418,  76352,  16795,  36836,  52584,  55492,
             60244,  69697,
            ...
             75089,  47856,  22409,  87155,  12194,   9381,  54193,  26517,
             42459,   4785],
           dtype='int64', length=1133)

In [25]:
#ratings.shape

In [26]:
ratings=ratings[ratings['User-ID'].isin(y)]

In [27]:
ratings.shape  ## top 608 people(who have given ratings to atleast 50 books) have given ratings to 140706 books.

(249388, 3)

In [28]:
ratings.head()

,User-ID,ISBN,Rating
173,276847,0446364193,0.0
174,276847,3257200552,5.0
175,276847,3379015180,0.0
176,276847,3404145909,8.0
177,276847,3404148576,8.0


Now let's merge this ratings dataframe to the books on the basis of ISBN.

In [29]:
ratings_with_books=ratings.merge(books,on="ISBN")
ratings_with_books

,User-ID,ISBN,Rating,Title,Author,Year,Publisher
0,276847,0446364193,0.0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books
1,278418,0446364193,0.0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books
2,5483,0446364193,0.0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books
3,7346,0446364193,0.0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books
4,8362,0446364193,0.0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books
...,...,...,...,...,...,...,...
108762,90423,0671877372,0.0,TOUCHED BY MAGIC,Doranna Durgin,1996,Baen
108763,90694,3596223946,0.0,"Sommerhaus, spÃ¤ter: ErzÃ¤hlungen (Collection ...",Judith Hermann,1998,S. Fischer
108764,90766,0330359916,0.0,Night Letters,Robert Dessaix,0,Pan Books Ltd
108765,90766,0330363239,6.0,Dirt music,Tim Winton,2001,Picador


In [30]:
number_ratings=ratings_with_books.groupby("Title")['Rating'].count().reset_index()
number_ratings

,Title,Rating
0,Beyond IBM: Leadership Marketing and Finance ...,1
1,It Takes Two,1
2,"Q-Space (Star Trek The Next Generation, Book 47)",7
3,Tales of Terror and Suspense,1
4,"' Sie belieben wohl zu scherzen, Mr. Feynman.'...",1
...,...,...
23170,stardust,10
23171,why I'm like this : True Stories,3
23172,"Â¿QuÃ© me quieres, amor?",1
23173,Ã?Â?ber die Freiheit.,1


In [31]:
number_ratings.rename(columns={"Rating":"No. of Ratings"},inplace=True)
number_ratings

,Title,No. of Ratings
0,Beyond IBM: Leadership Marketing and Finance ...,1
1,It Takes Two,1
2,"Q-Space (Star Trek The Next Generation, Book 47)",7
3,Tales of Terror and Suspense,1
4,"' Sie belieben wohl zu scherzen, Mr. Feynman.'...",1
...,...,...
23170,stardust,10
23171,why I'm like this : True Stories,3
23172,"Â¿QuÃ© me quieres, amor?",1
23173,Ã?Â?ber die Freiheit.,1


In [32]:
final_ratings=ratings_with_books.merge(number_ratings,on="Title")
final_ratings

,User-ID,ISBN,Rating,Title,Author,Year,Publisher,No. of Ratings
0,276847,0446364193,0.0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,61
1,278418,0446364193,0.0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,61
2,5483,0446364193,0.0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,61
3,7346,0446364193,0.0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,61
4,8362,0446364193,0.0,Along Came a Spider (Alex Cross Novels),James Patterson,1993,Warner Books,61
...,...,...,...,...,...,...,...,...
108762,90423,0671877372,0.0,TOUCHED BY MAGIC,Doranna Durgin,1996,Baen,1
108763,90694,3596223946,0.0,"Sommerhaus, spÃ¤ter: ErzÃ¤hlungen (Collection ...",Judith Hermann,1998,S. Fischer,1
108764,90766,0330359916,0.0,Night Letters,Robert Dessaix,0,Pan Books Ltd,1
108765,90766,0330363239,6.0,Dirt music,Tim Winton,2001,Picador,1


In [33]:
final_ratings.shape

(108767, 8)

In [34]:
final_ratings=final_ratings[final_ratings['No. of Ratings']>=25]
final_ratings.shape

(29601, 8)

In [35]:
final_ratings.drop_duplicates(["User-ID","Title"],inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
final_ratings.shape

(28924, 8)

In [37]:
book_pivot=final_ratings.pivot_table(columns="User-ID",index="Title",values="Rating")
book_pivot

User-ID,183,243,254,507,626,638,643,741,882,929,1025,1211,1424,1435,1674,1733,1848,1903,2030,2033,2110,2179,2276,2358,2766,2891,2977,3145,3167,3346,3363,3371,3373,3538,3556,3675,3757,3827,4017,4092,...,89014,89093,89179,89192,89207,89307,89361,89391,89551,89602,89867,89869,90049,90137,90232,90286,90391,90423,90616,90694,90766,276847,276925,276929,277157,277195,277378,277427,277478,277639,277928,277965,278026,278137,278144,278188,278418,278582,278633,278843
Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1984,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,0.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Word Freak: Heartbreak, Triumph, Genius, and Obsession in the World of Competitive Scrabble Players",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wuthering Heights,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Year of Wonders,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
book_pivot.shape

(671, 1054)

In [39]:
book_pivot.fillna(0,inplace=True)
book_pivot

User-ID,183,243,254,507,626,638,643,741,882,929,1025,1211,1424,1435,1674,1733,1848,1903,2030,2033,2110,2179,2276,2358,2766,2891,2977,3145,3167,3346,3363,3371,3373,3538,3556,3675,3757,3827,4017,4092,...,89014,89093,89179,89192,89207,89307,89361,89391,89551,89602,89867,89869,90049,90137,90232,90286,90391,90423,90616,90694,90766,276847,276925,276929,277157,277195,277378,277427,277478,277639,277928,277965,278026,278137,278144,278188,278418,278582,278633,278843
Title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1984,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Word Freak: Heartbreak, Triumph, Genius, and Obsession in the World of Competitive Scrabble Players",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Wuthering Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
from scipy.sparse import csr_matrix
book_sparse=csr_matrix(book_pivot)
type(book_sparse)

scipy.sparse.csr.csr_matrix

In [41]:
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors(algorithm="brute")
model.fit(book_sparse)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [73]:
distances, suggestions=model.kneighbors(book_pivot.iloc[202,:].values.reshape(1,-1),n_neighbors=6)

In [74]:
distances

array([[ 0.        , 56.93856338, 59.81638571, 61.66036004, 63.08724118,
        63.5295207 ]])

In [75]:
suggestions

array([[202, 205, 203, 268, 194, 608]])

In [76]:
for i in range(len(suggestions)):
  print(book_pivot.index[suggestions[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Like Water for Chocolate: A Novel in Monthly Installments With Recipes, Romances and Home Remedies',
       'Good Harbor: A Novel', 'The Woman Next Door'],
      dtype='object', name='Title')


In [72]:
book_pivot.index[54]
np.where(book_pivot.index=="Harry Potter and the Chamber of Secrets (Book 2)")[0][0]

202

Let's build a function, that when we input the name of a book, it gives us the recommendations of similar books.

In [81]:
def recommend_books(book_name):
  book_id=np.where(book_pivot.index==book_name)[0][0]
  distances,suggestions=model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=6)
  for i in range(len(suggestions)):
    print(book_pivot.index[suggestions[i]])

In [82]:
recommend_books("Ashes to Ashes")

Index(['Ashes to Ashes', 'Riptide', 'The First Wives Club Movie Tie In',
       'Mortal Fear', 'A Thin Dark Line (Mysteries &amp; Horror)',
       'The Woman Next Door'],
      dtype='object', name='Title')
